In [ ]:
%matplotlib inline

from utils.runner import RunsConfig, Runner
from utils.plotter import CanvasConfig
from traffic.environment import TrafficEnvironment
from traffic.agent import FixedCycleTrafficAgent, QLTrafficAgent, DQLTrafficAgent

canvas_config = CanvasConfig(
  [
    'system_total_stopped',
    'system_total_waiting_time',
    'system_mean_waiting_time',
    'system_mean_speed'
  ],
  1,
  100
)

traffic_env = TrafficEnvironment(
  net = '2way-single-intersection/2way-single-intersection.net.xml',
  rou = '2way-single-intersection/2way-single-intersection-2.rou.xml',
  seconds = 100000,
  delta_time = 10,
  yellow_time = 4,
  min_green = 10,
  max_green = 50
)

configs: list[RunsConfig] = [
  {
    'cls': FixedCycleTrafficAgent,
    'configs': [
      {
        'name': 'fixedcycle',
        'repeat': 1,
        'color': '#330000'
      }
    ]
  },
  {
    'cls': QLTrafficAgent,
    'configs': [
      {
        'name': 'ql_1',
        'repeat': 3,
        'color': '#eeff0a',
        'alpha': 0.2,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_2',
        'repeat': 3,
        'color': '#0a5eee',
        'alpha': 0.2,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_3',
        'repeat': 3,
        'color': '#0affb1',
        'alpha': 0.2,
        'gamma': 0.8,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_4',
        'repeat': 3,
        'color': '#ff0aff',
        'alpha': 0.2,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_5',
        'repeat': 3,
        'color': '#0affff',
        'alpha': 0.1,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_6',
        'repeat': 3,
        'color': '#00a5ff',
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_7',
        'repeat': 3,
        'color': '#a500ff',
        'alpha': 0.1,
        'gamma': 0.8,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_8',
        'repeat': 3,
        'color': '#a5a5a5',
        'alpha': 0.1,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      }
    ]
  },
  {
    'cls': DQLTrafficAgent,
    'configs': [
      {
        'name': 'dql_1',
        'repeat': 3,
        'color': '#ffea00',
        'alpha': 0.2,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_2',
        'repeat': 3,
        'color': '#0000ff',
        'alpha': 0.2,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_3',
        'repeat': 3,
        'color': '#fa7500',
        'alpha': 0.2,
        'gamma': 0.8,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_4',
        'repeat': 3,
        'color': '#0e0e0e',
        'alpha': 0.2,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_5',
        'repeat': 3,
        'color': '#bbff0e',
        'alpha': 0.1,
        'gamma': 0.6,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_6',
        'repeat': 3,
        'color': '#660eff',
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_7',
        'repeat': 3,
        'color': '#ff0033',
        'alpha': 0.1,
        'gamma': 0.8,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_8',
        'repeat': 3,
        'color': '#2eff33',
        'alpha': 0.1,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      }
    ]
  }
]

runner = Runner(canvas_config, traffic_env, configs)

runner.run(runner.learn(), 100000, False)